# Create Mock Energy Prices DataFrame

In [2]:
import pandas as pd
from datetime import timedelta
import numpy as np

# Load the CSV file with proper separator and handle encoding
df = pd.read_csv('/Users/juliusbecker/Project_Energy_Tariffs/data/training_dataset.csv', 
                 sep=';',  # Use semicolon separator
                 encoding='utf-8-sig',  # Handle BOM character
                 on_bad_lines='skip')

print(f"Successfully loaded with shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\nFirst few rows:")
print(df.head())

Successfully loaded with shape: (70176, 19)
Columns: ['Datum von', 'Datum bis', 'Deutschland/Luxemburg [€/MWh] Berechnete Auflösungen', '∅ Anrainer DE/LU [€/MWh] Berechnete Auflösungen', 'Belgien [€/MWh] Berechnete Auflösungen', 'Dänemark 1 [€/MWh] Berechnete Auflösungen', 'Dänemark 2 [€/MWh] Berechnete Auflösungen', 'Frankreich [€/MWh] Berechnete Auflösungen', 'Niederlande [€/MWh] Berechnete Auflösungen', 'Norwegen 2 [€/MWh] Berechnete Auflösungen', 'Österreich [€/MWh] Berechnete Auflösungen', 'Polen [€/MWh] Berechnete Auflösungen', 'Schweden 4 [€/MWh] Berechnete Auflösungen', 'Schweiz [€/MWh] Berechnete Auflösungen', 'Tschechien [€/MWh] Berechnete Auflösungen', 'DE/AT/LU [€/MWh] Berechnete Auflösungen', 'Italien (Nord) [€/MWh] Originalauflösungen', 'Slowenien [€/MWh] Berechnete Auflösungen', 'Ungarn [€/MWh] Berechnete Auflösungen']

First few rows:
          Datum von         Datum bis  \
0  01.01.2023 00:00  01.01.2023 00:15   
1  01.01.2023 00:15  01.01.2023 00:30   
2  01.01.2023 

In [3]:
# Process the date columns and move them up by one year
print("Processing dates...")

# Convert the German date format to datetime and add one year
df['Datum von'] = pd.to_datetime(df['Datum von'], format='%d.%m.%Y %H:%M')
df['Datum bis'] = pd.to_datetime(df['Datum bis'], format='%d.%m.%Y %H:%M')

# Add one year to both date columns
df['Datum von'] = df['Datum von'] + pd.DateOffset(years=1)
df['Datum bis'] = df['Datum bis'] + pd.DateOffset(years=1)

print(f"Date range after moving up one year:")
print(f"From: {df['Datum von'].min()} to {df['Datum von'].max()}")

# Show a few sample rows to verify
print("\nSample of updated data:")
print(df[['Datum von', 'Datum bis', 'Deutschland/Luxemburg [€/MWh] Berechnete Auflösungen']].head())

Processing dates...
Date range after moving up one year:
From: 2024-01-01 00:00:00 to 2025-12-31 23:45:00

Sample of updated data:
            Datum von           Datum bis  \
0 2024-01-01 00:00:00 2024-01-01 00:15:00   
1 2024-01-01 00:15:00 2024-01-01 00:30:00   
2 2024-01-01 00:30:00 2024-01-01 00:45:00   
3 2024-01-01 00:45:00 2024-01-01 01:00:00   
4 2024-01-01 01:00:00 2024-01-01 01:15:00   

  Deutschland/Luxemburg [€/MWh] Berechnete Auflösungen  
0                                              -5,17    
1                                              -5,17    
2                                              -5,17    
3                                              -5,17    
4                                              -1,07    


In [5]:
# Transform the data to match the expected API format
print("Transforming data to match expected API format...")

# Create a new dataframe with the expected column structure
price_data = pd.DataFrame()

# Use 'Datum von' as the datetime column
price_data['datetime'] = df['Datum von']

# Use the German electricity price as 'predicted_mean' 
# Convert German decimal format (5,17) to English format (5.17)
german_price_col = 'Deutschland/Luxemburg [€/MWh] Berechnete Auflösungen'
price_data['predicted_mean'] = df[german_price_col].astype(str).str.replace(',', '.').replace('-', '0')

# Convert to numeric, handling any errors
price_data['predicted_mean'] = pd.to_numeric(price_data['predicted_mean'], errors='coerce')

# Fill any NaN values with the mean price
avg_price = price_data['predicted_mean'].mean()
price_data['predicted_mean'] = price_data['predicted_mean'].fillna(avg_price)

# Add confidence intervals (optional - using simple estimates)
price_data['lower_ci'] = price_data['predicted_mean'] * 0.8  # 20% below
price_data['upper_ci'] = price_data['predicted_mean'] * 1.2  # 20% above

print(f"✅ Transformed data shape: {price_data.shape}")
print(f"📊 Columns: {list(price_data.columns)}")
print(f"💰 Price range: {price_data['predicted_mean'].min():.2f} - {price_data['predicted_mean'].max():.2f} €/MWh")
print(f"📈 Average price: {avg_price:.2f} €/MWh")

# Show sample of transformed data
print("\nSample of transformed data:")
print(price_data.head())

Transforming data to match expected API format...
✅ Transformed data shape: (70176, 4)
📊 Columns: ['datetime', 'predicted_mean', 'lower_ci', 'upper_ci']
💰 Price range: -500.00 - 936.28 €/MWh
📈 Average price: 86.83 €/MWh

Sample of transformed data:
             datetime  predicted_mean  lower_ci  upper_ci
0 2024-01-01 00:00:00           -5.17    -4.136    -6.204
1 2024-01-01 00:15:00           -5.17    -4.136    -6.204
2 2024-01-01 00:30:00           -5.17    -4.136    -6.204
3 2024-01-01 00:45:00           -5.17    -4.136    -6.204
4 2024-01-01 01:00:00           -1.07    -0.856    -1.284


In [7]:
# Resample to hourly averages and save the new mock price data
print("Resampling data to hourly averages...")

# Set datetime as index for resampling
price_data_hourly = price_data.set_index('datetime')

# Resample to hourly averages (15-minute intervals -> 1-hour intervals)
price_data_hourly = price_data_hourly.resample('H').mean()

# Reset index to get datetime back as a column
price_data_hourly = price_data_hourly.reset_index()

print(f"✅ Resampled data shape: {price_data_hourly.shape}")
print(f"🕐 Original 15-min intervals: {len(price_data)} → Hourly intervals: {len(price_data_hourly)}")
print(f"📅 Date range: {price_data_hourly['datetime'].min()} to {price_data_hourly['datetime'].max()}")

# Save the hourly data by overwriting the existing mock price data
output_path = '/Users/juliusbecker/Project_Energy_Tariffs/data/mock_price_data.csv'

# Save in standard CSV format (comma-separated, English decimal format)
price_data_hourly.to_csv(output_path, index=False)

print(f"💾 Successfully saved hourly mock price data to: {output_path}")

# Verify the file was saved correctly
import os
if os.path.exists(output_path):
    file_size = os.path.getsize(output_path) / (1024 * 1024)  # Size in MB
    print(f"📁 File size: {file_size:.2f} MB")
    print(f"📊 Data contains {len(price_data_hourly)} hourly price points")
    print(f"💰 Average hourly price: {price_data_hourly['predicted_mean'].mean():.2f} €/MWh")
else:
    print("❌ Error: File was not saved correctly")

# Show sample of final hourly data
print("\nSample of final hourly data:")
print(price_data_hourly.head())

Resampling data to hourly averages...
✅ Resampled data shape: (17544, 4)
🕐 Original 15-min intervals: 70176 → Hourly intervals: 17544
📅 Date range: 2024-01-01 00:00:00 to 2025-12-31 23:00:00
💾 Successfully saved hourly mock price data to: /Users/juliusbecker/Project_Energy_Tariffs/data/mock_price_data.csv
📁 File size: 0.80 MB
📊 Data contains 17544 hourly price points
💰 Average hourly price: 86.86 €/MWh

Sample of final hourly data:
             datetime  predicted_mean  lower_ci  upper_ci
0 2024-01-01 00:00:00           -5.17    -4.136    -6.204
1 2024-01-01 01:00:00           -1.07    -0.856    -1.284
2 2024-01-01 02:00:00           -1.47    -1.176    -1.764
3 2024-01-01 03:00:00           -5.08    -4.064    -6.096
4 2024-01-01 04:00:00           -4.49    -3.592    -5.388
